In [19]:
prompt = """
- Which role of the teacher would answer the following question better?
- The answer should be either 'math teacher' or 'history teacher' with no additional words.

Case 1
Question : what is the Newton’s law?
Answer : math teacher

Case 2 
Question : How did World War I end?
Answer : history teacher

Question : 
"""


In [25]:
import requests
import json

def send_chat_request(model, message_content):
    url = 'http://localhost:11434/api/chat'
    payload = {
        "model": model,
        "messages": [
            { "role": "user", "content": message_content }
        ]
    }

    response = requests.post(url, json=payload)
    
    response_lines = response.text.split('\n')
    contents = []
    
    for line in response_lines:
        if line.strip(): 
            try:
                message = json.loads(line)
                contents.append(message['message']['content'])
            except json.JSONDecodeError:
                print(f"Failed to decode JSON: {line}")
            except KeyError:
                print(f"Key error in line: {line}")
    
    concatenated_content = ''.join(contents)
    return concatenated_content

def make_question(question):
    return prompt + question

llama3 = "llama3"
question_1 = make_question("What is the Pythagorean theorem?")
question_2 = make_question("What were the main causes of the French Revolution?")

llama_answer_1 = send_chat_request(llama3, question_1)
llama_answer_2 = send_chat_request(llama3, question_2)
print(llama_answer_1)
print(llama_answer_2)

Math teacher
History teacher


In [26]:
phi3 = "phi3"
phi3_answer_1 = send_chat_request(llama3, question_1)
phi3_answer_2 = send_chat_request(llama3, question_2)
print(phi3_answer_1)
print(phi3_answer_2)

Math teacher
History teacher
